# Homework 2

## Imports

In [1]:
import nbtools
nbtools.setup_nb()

In [2]:
import itertools
import sympy
from sympy.diffgeom import Manifold, Patch
from pystein import coords, metric, curvature
from pystein.utilities import tensor_pow as tpow

## Problem 2

In [3]:
M = Manifold('M', dim=3)
P = Patch('origin', M)
r, theta, phi = sympy.symbols('r theta phi', nonnegative=True)
cs = coords.CoordSystem('spherical-polar', P, [r, theta, phi])
dr, dtheta, dphi = cs.base_oneforms()


ds2 = tpow(dr, 2) + r ** 2 * tpow(dtheta, 2) + r ** 2 * sympy.sin(theta) ** 2 * tpow(dphi, 2)
gamma = metric.Metric(twoform=ds2)

gamma

r**2*sin(theta)**2*TensorProduct(dphi, dphi) + r**2*TensorProduct(dtheta, dtheta) + TensorProduct(dr, dr)

In [11]:
christoffels = [((i, j, k), curvature.christoffel_symbol_component(i, j, k, gamma)) 
                for i, j, k in itertools.product(range(3), range(3), range(3))]

In [4]:
riccis = [((i, j), curvature.ricci_tensor_component(i, j, gamma)) 
          for i, j in itertools.product(range(3), range(3))]

In [12]:
curvature.display_components(christoffels)

In [8]:
riccis[0][-1]

Derivative((2*Derivative(0, r) - Derivative(1, theta))/(2*r**2), theta) - Derivative(Derivative(r**2, r)/(2*r**2), r) + Derivative((2*Derivative(0, r) - Derivative(1, phi))/(2*r**2*sin(theta)**2), phi) - Derivative(Derivative(r**2*sin(theta)**2, r)/(2*r**2*sin(theta)**2), r) - (2*Derivative(0, r) - Derivative(1, phi))*Derivative(1, phi)/(4*r**2*sin(theta)**2) - (2*Derivative(0, r) - Derivative(1, theta))*Derivative(1, theta)/(4*r**2) + Derivative(1, r)*Derivative(r**2, r)/(4*r**2) + Derivative(1, r)*Derivative(r**2*sin(theta)**2, r)/(4*r**2*sin(theta)**2) + (2*Derivative(0, r) - Derivative(1, phi))*Derivative(r**2, phi)/(4*r**4*sin(theta)**2) + (2*Derivative(0, r) - Derivative(1, phi))*Derivative(r**2*sin(theta)**2, phi)/(4*r**4*sin(theta)**4) + (2*Derivative(0, r) - Derivative(1, theta))*Derivative(r**2, theta)/(4*r**4) + (2*Derivative(0, r) - Derivative(1, theta))*Derivative(r**2*sin(theta)**2, theta)/(4*r**4*sin(theta)**2) - (-Derivative(0, phi) + Derivative(0, r) + Derivative(0, th

In [9]:
riccis[0][-1].doit()

0

In [5]:
curvature.display_components(riccis)